In [1]:
from nltk.corpus import reuters, brown  # nltk corpus
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
import random

In [2]:
corpus = reuters

In [3]:
first_sentence = corpus.sents()[1]
print(first_sentence)

['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.']


In [4]:
# Get the bigrams
#print(list(trigrams(first_sentence)))
# P(Reuter|'They', 'told')
tri = list(trigrams(first_sentence))
for t in tri:
    print(t)

('They', 'told', 'Reuter')
('told', 'Reuter', 'correspondents')
('Reuter', 'correspondents', 'in')
('correspondents', 'in', 'Asian')
('in', 'Asian', 'capitals')
('Asian', 'capitals', 'a')
('capitals', 'a', 'U')
('a', 'U', '.')
('U', '.', 'S')
('.', 'S', '.')
('S', '.', 'Move')
('.', 'Move', 'against')
('Move', 'against', 'Japan')
('against', 'Japan', 'might')
('Japan', 'might', 'boost')
('might', 'boost', 'protectionist')
('boost', 'protectionist', 'sentiment')
('protectionist', 'sentiment', 'in')
('sentiment', 'in', 'the')
('in', 'the', 'U')
('the', 'U', '.')
('U', '.', 'S')
('.', 'S', '.')
('S', '.', 'And')
('.', 'And', 'lead')
('And', 'lead', 'to')
('lead', 'to', 'curbs')
('to', 'curbs', 'on')
('curbs', 'on', 'American')
('on', 'American', 'imports')
('American', 'imports', 'of')
('imports', 'of', 'their')
('of', 'their', 'products')
('their', 'products', '.')


### build a trigram model

In [5]:
# creat all 
model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in corpus.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
 

In [6]:
# count of the bigrams preceded by price
print(model["today", "the"]["price"])
print(model["today", "the"]["nonexistingword"])
print(model[None, None]["The"])

2
0
8839


In [7]:
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

In [8]:
print(model["today", "the"]["price"])
print(model["today", "the"]["nonexistingword"])
print(model[None, None]["The"])

0.1111111111111111
0.0
0.16154324146501936


In [9]:
#model[None, None]

Now we have a trigram language model. Let’s generate some text

In [10]:
text = ["today", "the"] # store the sentence
sentence_finished = False
 
while not sentence_finished:
    r = random.random()
    accumulator = .0
 
    for word in model[tuple(text[-2:])].keys():
        accumulator += model[tuple(text[-2:])][word]
 
        if accumulator >= r:
            text.append(word)
            break
 
    # verify if sentence is completed
    if text[-2:] == [None, None]:
        sentence_finished = True


In [11]:
print(' '.join([t for t in text if t]))

today the overseas units and Kleinwort Benson chief financial officer , Jeffery Lane , to improve and the treasury bill investment is estimated to have sold mark bonds on the agenda .


## loading data

In [12]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from nltk import word_tokenize, sent_tokenize
import re
from nltk.tokenize import ToktokTokenizer
import os
import requests
import io 

In [13]:
word_tokenize(sent_tokenize("This is a foobar sentence. Yes it is.")[0])

['This', 'is', 'a', 'foobar', 'sentence', '.']

In [14]:
if os.path.isfile('data/lm_text.txt'):
    with io.open('data/lm_text.txt', encoding='utf8') as fin:
        text = fin.read()

In [15]:
# Tokenize the text.
tokenized_text = [list(map(str.lower, word_tokenize(sent))) 
                  for sent in sent_tokenize(text)]

In [16]:
#tokenized_text[0]

In [17]:
# Preprocess the tokenized text for 3-grams language modelling
n = 3 # n gram
train_data, padded_sents = padded_everygram_pipeline(n, tokenized_text)

### Training an tri-gram Model

In [18]:
# initialize MLE model - MLE model, creates an empty vocabulary
model = MLE(n)

In [19]:
len(model.vocab)

0

In [20]:
# traine model 
model.fit(train_data, padded_sents)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 1391 items>


In [21]:
len(model.vocab)

1391

In [22]:
print(model.vocab.lookup(tokenized_text[0]))

('language', 'is', 'never', ',', 'ever', ',', 'ever', ',', 'random', 'adam', 'kilgarriff', 'abstract', 'language', 'users', 'never', 'choose', 'words', 'randomly', ',', 'and', 'language', 'is', 'essentially', 'non-random', '.')


In [23]:
# the model automatically replace words not in the vocabulary with `<UNK>`.
print(model.vocab.lookup('language is never random unseenword .'.split()))

('language', 'is', 'never', 'random', '<UNK>', '.')


### Exploring the model

In [24]:
print(model.counts)

<NgramCounter with 3 ngram orders and 19611 ngrams>


In [25]:
phrase = 'language'
print("number of bigrams for the phrase", phrase, model.counts[phrase])

number of bigrams for the phrase language 25


In [26]:
# number of bigrames of language ffollowed y us
print("number of bigrams for the phrase", model.counts[['language']]['is'])

number of bigrams for the phrase 11


In [27]:
print("number of bigrams for the phrase", model.counts[['language', 'is']]['never'])

number of bigrams for the phrase 7


In [28]:
# what is the probability score for the word language i.e how likely s the word in our vocanulary
model.score('language') # P('language')

0.003691671588895452

In [29]:
# probability of 'is' given 'language'
model.score('is', 'language'.split())  # P('is'|'language')

0.44

In [30]:
# P('never'|'language is')
model.score('never', 'language is'.split())  

0.6363636363636364

In [31]:
model.logscore("never", "language is".split())

-0.6520766965796932

### Generating text

In [32]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [33]:
detokenize = TreebankWordDetokenizer().detokenize

def generate_sent(model, num_words, random_seed=42):
    """Generate text method  """
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [34]:
generate_sent(model, 20, random_seed=7)

'and carroll used hypothesis testing has been used, and a half.'